# Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

In [322]:
import pandas as pd
import numpy as np

## 1. Import Neighborhoods in Toronto from Wikipedia html page

In [323]:
# Import dataset of Neighborhood of Toronto from Wikipedia
import requests
url_wikiToronto = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_wikiToronto = requests.get(url_wikiToronto)

In [324]:
from bs4 import BeautifulSoup

# Parse the HMTL as a string
soup = BeautifulSoup(html_wikiToronto.text, 'html.parser')
#print(soup.prettify())

# Grab the first table 
table = soup.find_all('table')[0]

In [325]:
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [326]:
# Find Headers
column_names = []
row_header = table.find_all('tr')[0]

# Handle column names if we find them
th_tags = row_header.find_all('th') 
if len(th_tags) > 0 and len(column_names) == 0:
    for th in th_tags:
        column_names.append(th.get_text())

columns = column_names if len(column_names) > 0 else range(0,n_columns)
columns

['Postal Code\n', 'Borough\n', 'Neighbourhood\n']

In [327]:
# Find number of row
row_count = 0
for row in table.find_all('tr'):
    row_count += 1
    
print ("Table has ", row_count," lines.")

Table has  181  lines.


In [328]:
# Find number of column
col_count = 0
for col in table.find_all('th'):
    col_count += 1
    
print ("Table has", col_count,' columns')

Table has 3  columns


In [329]:
# Parse the content of Table Tag into Dataframe                    
# Iniate empty Table
new_table = pd.DataFrame(columns=columns, index=range(0,row_count))

# Parse Tag Table in to the Table
row_marker = 0
for row in table.find_all('tr'):
        
        column_marker = 0
        
        # Otherwise just retrieve content of the columns
        columns = row.find_all('td')
        for column in columns:
            new_table.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        row_marker += 1

new_table.head()

,Postal Code,Borough,Neighbourhood
0,NaN,NaN,NaN
1,M1A\n,Not assigned\n,Not assigned\n
2,M2A\n,Not assigned\n,Not assigned\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n


## 2. Datacleaning : filter the dataset to get only valuable data for analysis

In [330]:
# Remove \n in header
new_table = new_table.rename(columns={"Postal Code\n":"Postal Code"})
new_table = new_table.rename(columns={"Borough\n":"Borough"})
new_table = new_table.rename(columns={"Neighbourhood\n":"Neighbourhood"})

new_table.head()

,Postal Code,Borough,Neighbourhood
0,NaN,NaN,NaN
1,M1A\n,Not assigned\n,Not assigned\n
2,M2A\n,Not assigned\n,Not assigned\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n


In [331]:
# Remove \n in the text fields
new_table = new_table.replace('\n','', regex=True)
new_table.head()

,Postal Code,Borough,Neighbourhood
0,NaN,NaN,NaN
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [332]:
new_table = new_table.dropna()
new_table.head()

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [333]:

df = new_table
df.head()

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## 3. Datawrangling : merge duplicates

In [334]:
# Find Duplicates
df.groupby(['Borough']).count()

,Postal Code,Neighbourhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Not assigned,77,77
Scarborough,17,17


In [335]:
# Find Duplicates
df.groupby(['Postal Code']).count()

,Borough,Neighbourhood
Postal Code,,
M1A,1,1
M1B,1,1
M1C,1,1
M1E,1,1
M1G,1,1
M1H,1,1
M1J,1,1
M1K,1,1
M1L,1,1


In [336]:
## Datawrangling : re-assign for missing values
#df_M5A = pd.DataFrame(columns=columns, index=range(0,row_count))
df_M5A = df.loc[lambda df: df['Postal Code'] == 'M5A']
df_M5A.head()

,Postal Code,Borough,Neighbourhood
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [337]:
print ("Duplicates in Postal Code :  Case not occur.")

Duplicates in Postal Code :  Case not occur.


In [338]:
# Neighborhood NotAssigned, Borough Assigned
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_Neighbourhood_NA = df.loc[lambda df: df['Neighbourhood'] == 'Not assigned']
df_nn = df_Neighbourhood_NA

df_Borough_NA = df_nn.loc[lambda df_nn: df_nn['Borough'] == 'Not assigned']
df_Borough_A = df_nn.loc[lambda df_nn: df_nn['Borough'] != 'Not assigned']

df_Borough_A
print ("Missing Neighborhood : Case not occur.")

Missing Neighborhood : Case not occur.


In [339]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
df_clean = df.loc[new_table['Borough']!='Not assigned']

In [340]:
df_clean.head()

,Postal Code,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [341]:
df_clean.shape

(103, 3)

# 4. Adding Coordinates in the dataframe

In [313]:
# Import Coordinate related to Postal Code
gc = pd.read_csv("http://cocl.us/Geospatial_data")
gc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [342]:
df_geo = pd.merge(df_clean, gc, 
                  #left_on=df_clean['Postal Code'], right_on = gc['Postal Code'], 
                  how='left',
                  on ='Postal Code'
                 )
df_geo.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
